# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GraphRepresentationGenerator** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from graph_representation_generator.graph_representation_generator import (
    GraphRepresentationGenerator,
)
from dataset_manager.movie_lens_manager import MovieLensManager
from dataset_manager.kg_manager import ROOT
from llm_manager.graph_prompter_hf.classifier import GraphPrompterHF

In [2]:
MODEL_NAME = "google/bert_uncased_L-2_H-128_A-2"
EPOCHS = 2
BATCH_SIZE_LLM = 256

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [3]:
kg_manager = MovieLensManager()

llm_df = kg_manager.llm_df.merge(kg_manager.target_df[["id", "prompt_feature_title", "prompt_feature_genres"]].rename(columns={"id": "target_id"}), on = "target_id")
llm_df

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-32m.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [4]:
kg_manager.data

HeteroData(
  source={ node_id=[200948] },
  target={
    node_id=[87585],
    x=[87585, 20],
  },
  (source, edge, target)={ edge_index=[2, 32000204] },
  (target, rev_edge, source)={ edge_index=[2, 32000204] }
)

Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
graph_representation_generator_graph_prompter_hf = GraphRepresentationGenerator(
    kg_manager.data.to("cuda"),
    kg_manager.gnn_train_data.to("cuda"),
    kg_manager.gnn_val_data.to("cuda"),
    kg_manager.gnn_test_data.to("cuda"),
    hidden_channels=128,
    kge_dimension=128,
    force_recompute=False,
    device="cuda",
)

loading pretrained model
Device: 'cuda'


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [ ]:
graph_prompter_hf_bert_classifier = GraphPrompterHF(
    kge_manager=kg_manager,
    get_embeddings_cb=graph_representation_generator_graph_prompter_hf.get_embeddings,
    model_name=MODEL_NAME,
    root_path=f"{ROOT}/llm/graph_prompter_hf_frozen",
    vanilla_model_path=f"{ROOT}/llm/vanilla/training/best",
    # gnn_parameters=list(
    #     graph_representation_generator_graph_prompter_hf.model.parameters()
    # ),
)

Some weights of GraphPrompterHFBertForSequenceClassification were not initialized from the model checkpoint at ./data/llm/vanilla/training/best and are newly initialized because the shapes did not match:
- bert.embeddings.token_type_embeddings.weight: found shape torch.Size([4, 128]) in the checkpoint and torch.Size([6, 128]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device cuda
6
6
6


In [7]:
dataset_embedding = kg_manager.generate_graph_prompter_hf_embedding_dataset(
    graph_prompter_hf_bert_classifier.tokenizer.sep_token,
    graph_prompter_hf_bert_classifier.tokenizer.pad_token,
    graph_prompter_hf_bert_classifier.tokenize_function,
    # get_embeddings_cb=graph_representation_generator_graph_prompter_hf.get_embeddings,
)

Loading dataset from disk:   0%|          | 0/127 [00:08<?, ?it/s]

Loading dataset from disk:   0%|          | 0/46 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/46 [00:00<?, ?it/s]

In [8]:
graph_prompter_hf_bert_classifier.train_model_on_data(
    dataset_embedding,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE_LLM,
)
# graph_representation_generator_graph_prompter_hf.save_model()

  0%|          | 0/140002 [00:00<?, ?it/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.3224, 'grad_norm': 8.543179512023926, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}
{'loss': 0.3276, 'grad_norm': 4.8039774894714355, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}
{'loss': 0.2991, 'grad_norm': 5.412833213806152, 'learning_rate': 3e-06, 'epoch': 0.0}
{'loss': 0.2954, 'grad_norm': 2.7199974060058594, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.2791, 'grad_norm': 1.776186466217041, 'learning_rate': 5e-06, 'epoch': 0.0}
{'loss': 0.2537, 'grad_norm': 2.3738412857055664, 'learning_rate': 6e-06, 'epoch': 0.0}
{'loss': 0.271, 'grad_norm': 1.8420521020889282, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.2594, 'grad_norm': 1.443666934967041, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.2372, 'grad_norm': 1.860097885131836, 'learning_rate': 9e-06, 'epoch': 0.0}
{'loss': 0.2252, 'grad_norm': 1.4874489307403564, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 0.2476, 'grad_norm': 1.673872232437

  0%|          | 0/25001 [00:00<?, ?it/s]

{'eval_loss': 0.08122512698173523, 'eval_accuracy': 0.9732567296454397, 'eval_runtime': 16560.9945, 'eval_samples_per_second': 386.453, 'eval_steps_per_second': 1.51, 'epoch': 1.0}
{'loss': 0.0651, 'grad_norm': 0.411241739988327, 'learning_rate': 2.5086378689911257e-05, 'epoch': 1.0}
{'loss': 0.06, 'grad_norm': 0.4435168504714966, 'learning_rate': 2.5082794511906642e-05, 'epoch': 1.0}
{'loss': 0.067, 'grad_norm': 0.4343034029006958, 'learning_rate': 2.5079210333902027e-05, 'epoch': 1.0}
{'loss': 0.0495, 'grad_norm': 0.42155691981315613, 'learning_rate': 2.507562615589741e-05, 'epoch': 1.0}
{'loss': 0.0705, 'grad_norm': 0.3964107036590576, 'learning_rate': 2.507204197789279e-05, 'epoch': 1.0}
{'loss': 0.0577, 'grad_norm': 0.49210166931152344, 'learning_rate': 2.5068457799888172e-05, 'epoch': 1.0}
{'loss': 0.0737, 'grad_norm': 0.5480309128761292, 'learning_rate': 2.506487362188356e-05, 'epoch': 1.0}
{'loss': 0.058, 'grad_norm': 0.45785823464393616, 'learning_rate': 2.5061289443878942e-05

  0%|          | 0/25001 [00:00<?, ?it/s]

{'eval_loss': 0.08058305084705353, 'eval_accuracy': 0.9732726670458309, 'eval_runtime': 17347.5938, 'eval_samples_per_second': 368.93, 'eval_steps_per_second': 1.441, 'epoch': 2.0}
{'train_runtime': 141025.2852, 'train_samples_per_second': 254.14, 'train_steps_per_second': 0.993, 'train_loss': 0.06996742957570386, 'epoch': 2.0}


In [11]:
import torch

tens_1 = graph_representation_generator_graph_prompter_hf.model.to(
    device="cpu"
).state_dict()

tens_2 = torch.load("./data/gnn/backup/model_128.pth")
same = True
for key in tens_1.keys():
    if isinstance(tens_1[key], torch.Tensor):
        if not (tens_1[key] == tens_2[key]).all():
            same = False
print(same)

True


In [ ]:
dataset_embedding

DatasetDict({
    train: Dataset({
        features: ['source_id', 'target_id', 'split', 'labels', 'prompt_feature_title', 'prompt_feature_genres', 'prompt', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 17920114
    })
    val: Dataset({
        features: ['source_id', 'target_id', 'split', 'labels', 'prompt_feature_title', 'prompt_feature_genres', 'prompt', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 6400040
    })
    test: Dataset({
        features: ['source_id', 'target_id', 'split', 'labels', 'prompt_feature_title', 'prompt_feature_genres', 'prompt', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 6400040
    })
})